__Opened:__ Saturday, 3 January 2026, 11:14 AM
__Due:__ Monday, 20 April 2026, 12:14 PM
### Assignment
look at the the page

Deck of Cards API    https://deckofcardsapi.com/

This is an API that simulates dealing a deck of cards

Write a program that "deals" (prints out) 5 cards

first you need to shuffle

https://deckofcardsapi.com/api/deck/new/shuffle/?deck_count=1

get the deck_id, 

with the deck_id you can get the cards

https://deckofcardsapi.com/api/deck/<<deck_id>>/draw/?count=2

This example gets two cards

From there you need to print the value and the suit of each card.

save the file as assignment2-carddraw.py (or as a notebook)

Last few marks:
Check if the user has drawn a pair, triple, straight, or all of the same suit and congratulate the user.
